# Classical numerics and operator learning for Burgers' PDE with FDM and FEM

# Setup

We consider the periodic burgers equation in 1d:
$$
    \partial_t u (t, x)
=
    \nu (\partial_{x, x} u)(t, x) - \frac{(\partial_x u^2)(t, x)}{2}
=
    \nu (\partial_{x, x} u)(t, x) - (\partial_x u)(t, x) u(t, x),
$$
$$
    u(t, 0) = u(t, S),
\qquad
    (\partial_x u)(t, 0) = (\partial_x u)(t, S)
$$
for $(t, x) \in [0,T] \times [0, S]$.

We want to approximate the map
$$
\Phi(u(0, \cdot)) = u(T, \cdot).
$$

Problem parameters:  $T, S, \nu \in (0,\infty)$ and distribution of initial value

In [ ]:
import sys
import time
# from scipy.fft import fft, ifft, fftfreq
import numpy as np
import matplotlib.pyplot as plt
# import pandas as pd
import importlib
import torch
import openpyxl
import os
import json
import seaborn as sns

sys.path.insert(1, '../1_Modules')

# Importing the modules
import random_function_generators
import ode_methods
import training
import training_samples_generators
import operator_learning_models
import utils
import burgers_classical_methods
import evaluation_utils
import documentation_utils
import PDE_operations

In [ ]:
# Reloading the modules
importlib.reload(random_function_generators)
importlib.reload(ode_methods)
importlib.reload(training)
importlib.reload(training_samples_generators)
importlib.reload(utils)
importlib.reload(operator_learning_models)
importlib.reload(burgers_classical_methods)
importlib.reload(evaluation_utils)
importlib.reload(documentation_utils)
importlib.reload(PDE_operations)

from random_function_generators import *
from ode_methods import *
from training import *
from training_samples_generators import *
from operator_learning_models import *
from utils import *
from burgers_classical_methods import *
from evaluation_utils import *
from documentation_utils import *
from PDE_operations import *

In [ ]:
test_run = True

#Problem setup periodic Burgers PDE in 1d
###################################################
T = 1.
space_size = 2 * np.pi
laplace_factor = 0.1
dim = 1 # Dimension can only be 1 for Burgers

# initial value
var = 1000
decay_rate = 3.
offset = np.power(var, 1/decay_rate)
inner_decay = 2.
initial_value_generator = RandnFourierSeriesGenerator([var, decay_rate, offset, inner_decay, space_size, 1])

In [ ]:
# Discretization operations
x_values = x_values_periodic
reduce_dimension = reduce_dimension_periodic
get_higher_nr_spacediscr = get_higher_nr_spacediscr_periodic
create_boundary_values = create_boundary_values_periodic

In [ ]:
# Quantities derived from setup
###################################################
# Name of the PDE
pde_name = f"Burgers_T{T}_S{space_size}_nu{laplace_factor}_var{var}_decay{decay_rate}_offset{offset}_innerdecay{inner_decay}"

#Create folder for all outputs
output_folder_dir = create_output_folder(pde_name)

#Prepare df to store data
methods_data = pd.DataFrame(columns=["nr_params", "training_time", "test_time", "L2_error", "done_trainsteps", "learning_rate_history", "batch_size_history"])
methods = {}

# Set random seeds
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)

### Create Train and Test sets

In [ ]:
generate_data = True
data_load_folder = "2023-06-29 21h55m Code output/"

#Nr of input points allowed to be used by methods
nr_spacediscr = 128

#Method for reference solutions for training of models
reference_algorithm = lambda initial_values, nr_timesteps: burgers_spectral_freqsp_lirk(initial_values, T, laplace_factor, space_size, nr_timesteps)

# Train set parameters
train_space_resolution_step = 4 if test_run else 4
train_nr_timesteps = 1000 if test_run else 1000
nr_train_samples = 2**18 if test_run else 2**18
nr_validation_samples = 2**14 if test_run else 2**14

# Test set parameters
test_space_resolution_step = 8 if test_run else 8
test_nr_timesteps = 1500 if test_run else 1500
nr_test_samples = 2**14 if test_run else 2**14

only_save_rough = True

# Set the device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

parameters = {
    'T': T,
    'space_size': space_size,
    'laplace_factor': laplace_factor,
    'var': var,
    'decay_rate': decay_rate,
    'offset': offset,
    'inner_decay': inner_decay,
    'nr_spacediscr': nr_spacediscr,
    'train_space_resolution_step': train_space_resolution_step,
    'train_nr_timesteps': train_nr_timesteps,
    'nr_train_samples': nr_train_samples,
    'nr_validation_samples': nr_validation_samples,
    'test_space_resolution_step': test_space_resolution_step,
    'nr_test_samples': nr_test_samples,
    'test_nr_timesteps': test_nr_timesteps,
    'reference_algorithm': reference_algorithm.__name__,
    'only_save_rough': only_save_rough
}

# save parametesr
with open(output_folder_dir + 'train_test_parameters.json', 'w') as fp:
    json.dump(parameters, fp)

In [ ]:
# Produce train and test data
train_nr_spacediscr = get_higher_nr_spacediscr(nr_spacediscr, train_space_resolution_step)
test_nr_spacediscr = get_higher_nr_spacediscr(nr_spacediscr, test_space_resolution_step)

print("Generating train samples")
train_initial_values_fine, train_ref_sol_fine, train_initial_values_rough, train_ref_sol_rough = (
    get_data(
        initial_value_generator, reference_algorithm, 
        nr_train_samples, train_nr_spacediscr, train_nr_timesteps, 
        reduce_dimension, train_space_resolution_step, 'train', 
        output_folder_dir, generate_data, data_load_folder, parameters, only_save_rough
    ))
training_samples_generator = TrainingSamplesGeneratorFromSolutions(train_initial_values_rough, train_ref_sol_rough)

print("Generating validation samples")
validation_initial_values_fine, validation_ref_sol_fine, validation_initial_values_rough, validation_ref_sol_rough = (
    get_data(
        initial_value_generator, reference_algorithm, 
        nr_validation_samples, test_nr_spacediscr, test_nr_timesteps, 
        reduce_dimension, test_space_resolution_step, 'validate', 
        output_folder_dir, generate_data, data_load_folder, parameters, only_save_rough
    ))

print("Generating test samples")
test_initial_values_fine, test_ref_sol_fine, test_initial_values_rough, test_ref_sol_rough = (
    get_data(
        initial_value_generator, reference_algorithm, 
        nr_test_samples, test_nr_spacediscr, test_nr_timesteps, 
        reduce_dimension, test_space_resolution_step, 'test', 
        output_folder_dir, generate_data, data_load_folder, parameters, only_save_rough
    ))

In [ ]:
#Plot some reference solutions 
plot_reference_solutions(train_initial_values_rough, train_ref_sol_rough, 4, dim, x_values, space_size, pde_name, output_folder_dir)

# Create models and methods to be tested

In [ ]:
# Optimizer
optimizer_class = torch.optim.Adam
# Loss function
loss_fn = torch.nn.MSELoss()

### Operator learning models

In [ ]:
# Training hyperparams
OL_training_kwargs = {
    "max_trainsteps": 100000 if test_run else 100000,
    "initial_batchsize": 2**9 if test_run else 2**10,
    "max_batchsize":2**9 if test_run else 2**10,
    "output_steps": 200 if test_run else 200,
    "eval_steps": 400 if test_run else 400,
    "improvement_tolerance": 0.96 if test_run else 0.96,
    "initial_lr": 0.001
}
nr_runs = 1 if test_run else 5

In [ ]:
#ANN models
ann_foldername = output_folder_dir + "Results_ANN"

#ANN Parameters [layer_dims]
list_ann_params = [
    [[nr_spacediscr, 2**8, 2**9, 2**8, nr_spacediscr]],
    [[nr_spacediscr, 2**8, 2**10, 2**10, 2**8, nr_spacediscr]],
    [[nr_spacediscr, 2**9, 2**11, 2**11, 2**9, nr_spacediscr]],
    [[nr_spacediscr, 2**10, 2**12, 2**13, 2**12, 2**10, nr_spacediscr]]
]

create_and_train_models(
    modelclass = ANNModel, 
    list_params = list_ann_params,
    training_samples_generator = training_samples_generator,
    optimizer_class = optimizer_class,
    loss_fn = loss_fn,
    training_kwargs=OL_training_kwargs,
    lr_search_params=None,
    nr_runs = nr_runs,
    methods = methods,
    methods_data = methods_data,
    foldername = ann_foldername,
    test_input_values = test_initial_values_rough,
    test_ref_sol = test_ref_sol_rough,
    validation_input_values = validation_initial_values_rough,
    validation_ref_sol = validation_ref_sol_rough,
    pde_name = pde_name,
    local_learning_rates=False
)

In [ ]:
# CNN perdiodic models
CNNPeriodic_foldername = output_folder_dir + "Results_Periodic CNN"

#CNN Parameters [channel_dims, kernel_sizes, dim]
list_cnn_periodic_params = [
    [[1, 50, 50, 1],[51, 51, 51], 1],
    [[1, 50, 50, 50, 1],[41, 41, 41, 41], 1],
    [[1, 50, 100, 100, 50, 1], [31, 31, 31, 31, 31], 1],
    [[1, 100, 200, 200, 100, 100, 1], [31, 31, 31, 31, 31, 31], 1]
]

create_and_train_models(
    modelclass = CNNPeriodicnDModel, 
    list_params = list_cnn_periodic_params,
    training_samples_generator = training_samples_generator,
    optimizer_class = optimizer_class,
    loss_fn = loss_fn,
    training_kwargs=OL_training_kwargs,
    lr_search_params=None,
    nr_runs = nr_runs,
    methods = methods,
    methods_data = methods_data,
    foldername = CNNPeriodic_foldername,
    test_input_values = test_initial_values_rough,
    test_ref_sol = test_ref_sol_rough,
    validation_input_values = validation_initial_values_rough,
    validation_ref_sol = validation_ref_sol_rough,
    pde_name = pde_name,
    local_learning_rates=False
)

In [ ]:
# CNN simple enc dec models
CNNEncDec_foldername = output_folder_dir + "Results_Enc-Dec CNN"

#CNN Parameters [channel_dims, kernel_sizes, dim]
list_cnn_enc_dec_params = [
    [[1, 8, 32, 64],[2, 4, 2], 1],
    [[1, 8, 32, 64, 128], [4, 2, 2, 4], 1],
    [[1, 8, 32, 64, 128, 256], [4, 2, 2, 4, 2], 1],
    [[1, 8, 32, 64, 128, 256, 512, 512],[2, 2, 2, 2, 2, 2, 2], 1]
]

create_and_train_models(
    modelclass = CNNEncDec, 
    list_params = list_cnn_enc_dec_params,
    training_samples_generator = training_samples_generator,
    optimizer_class = optimizer_class,
    loss_fn = loss_fn,
    training_kwargs=OL_training_kwargs,
    lr_search_params=None,
    nr_runs = nr_runs,
    methods = methods,
    methods_data = methods_data,
    foldername = CNNEncDec_foldername,
    test_input_values = test_initial_values_rough,
    test_ref_sol = test_ref_sol_rough,
    validation_input_values = validation_initial_values_rough,
    validation_ref_sol = validation_ref_sol_rough,
    pde_name = pde_name,
    local_learning_rates=False
)

In [ ]:
# FNO models
FNO_foldername = output_folder_dir + "Results_FNO"

#list is [#modes, width, depth, dim]
list_fno_params = [
    [8, 20, 4, 1],
    [16, 20, 4, 1],
    [16, 30, 4, 1],
    [16, 30, 5, 1]
]

create_and_train_models(
    modelclass = FNOnDModel, 
    list_params = list_fno_params,
    training_samples_generator = training_samples_generator,
    optimizer_class = optimizer_class,
    loss_fn = loss_fn,
    training_kwargs=OL_training_kwargs,
    lr_search_params=None,
    nr_runs = nr_runs,
    methods = methods,
    methods_data = methods_data,
    foldername = FNO_foldername,
    test_input_values = test_initial_values_rough,
    test_ref_sol = test_ref_sol_rough,
    validation_input_values = validation_initial_values_rough,
    validation_ref_sol = validation_ref_sol_rough,
    pde_name = pde_name,
    local_learning_rates=False
)

In [ ]:
# DeepONets
DeepONet_foldername = output_folder_dir + "Results_DeepONet"

DeepONet_space_grid = np.transpose(x_values(nr_spacediscr, space_size)) # Need shape (nr_spacediscr, dim)

# DeepOnet Parameters [trunk_architecture, branch_architecture, eval_points]
list_deeponet_params = [
    [[nr_spacediscr, 200, 200, 100],[1, 50, 100], DeepONet_space_grid],
    [[nr_spacediscr, 200, 200, 300],[1, 50, 100, 300], DeepONet_space_grid],
    [[nr_spacediscr, 300, 600, 1000, 500],[1, 100, 300, 500], DeepONet_space_grid],
    [[nr_spacediscr, 300, 600, 2000, 1000],[1, 200, 500, 1000], DeepONet_space_grid]
]


create_and_train_models(
    modelclass = DeepONet, 
    list_params = list_deeponet_params,
    training_samples_generator = training_samples_generator,
    optimizer_class = optimizer_class,
    loss_fn = loss_fn,
    training_kwargs=OL_training_kwargs,
    lr_search_params=None,
    nr_runs = nr_runs,
    methods = methods,
    methods_data = methods_data,
    foldername = DeepONet_foldername,
    test_input_values = test_initial_values_rough,
    test_ref_sol = test_ref_sol_rough,
    validation_input_values = validation_initial_values_rough,
    validation_ref_sol = validation_ref_sol_rough,
    pde_name = pde_name,
    local_learning_rates=False
)

### Classical Methods

In [ ]:
# Discretization parameters
nr_timesteps_fdm = [5, 10, 15, 20] # [1, 5, 7, 8, 9, 10, 15, 20] # , 30, 40, 70, 100, 200, 300, 400, 500, 1000, 1500, 2000]
nr_timesteps_fem = [5, 10, 15, 20] # ,, 100]
nr_timesteps_spe = [5, 10, 15, 20] # , 50, 60, 70, 100, 200, 300, 400, 500, 600, 700, 800, 900, 1000]

#Create all methods for the correponding timesteps
# FDM (I do not evaluate the euler method at the moment)
for nr_timesteps in nr_timesteps_fdm:
    for conservative in [True]: #[True, False]:
        for v in [2]: #range(3):
            # name = "FDM (%i CN tsps, conservative:%i, v:%i)"% (nr_timesteps, conservative, v)
            name = f"FDM ({nr_timesteps} Crank Nicolson timesteps)"
            methods[name] = lambda initial_values_rough, nr_timesteps=nr_timesteps, v=v, conservative=conservative: burger_fdm_lirk(initial_values_rough, T, laplace_factor, space_size, nr_timesteps, v, conservative)
            methods_data.at[name, "training_time"] = 0

# FEM
for nr_timesteps in nr_timesteps_fem:
    # name = "FEM (%i CN tsps)" % nr_timesteps
    name = f"FEM ({nr_timesteps} Crank Nicolson timesteps)"
    methods[name] = lambda initial_values_rough, nr_timesteps=nr_timesteps: burger_fem_lirk(initial_values_rough, T, laplace_factor, space_size, nr_timesteps)
    methods_data.at[name, "training_time"] = 0

# Spectral eval (I do not evaluate the euler method at the moment)
for nr_timesteps in nr_timesteps_spe:
    for conservative in [True]: #[True, False]:
        # name = "Spectral (%i CN tsps, conservative:%i)" % (nr_timesteps, conservative)
        name = f"Spectral ({nr_timesteps} Crank Nicolson timesteps)"
        methods[name] = lambda initial_values_rough, nr_timesteps=nr_timesteps, conservative=conservative: burgers_spectral_freqsp_lirk(initial_values_rough, T, laplace_factor, space_size, nr_timesteps, conservative)
        methods_data.at[name, "training_time"] = 0

In [ ]:
# sys.path.insert(1, '../3_ADANNs/1_ADANN_Modules')

# from ADANNs import *

# #Create all methods for the correponding timesteps
# # FDM (I do not evaluate the euler method at the moment)
# for nr_timesteps in nr_timesteps_fdm:
#     for v in [2]: 
#         name = f"ADANNs (FDM, {nr_timesteps} Crank-Nicolson time steps)"
#         methods[name] = SecondOrderLirkFDMPeriodicBurgersAdannBasemodel_learnFirstOrder(T, laplace_factor, space_size, nr_spacediscr, nr_timesteps, v).to(device)
#         methods_data.at[name, "training_time"] = 0

# Evaluation

In [ ]:
# Evaluate all the methods and create plots
nr_of_eval_runs = 1000 if test_run else 1000
plot_histogram = False if test_run else True

method_categories = ["ANN", "Periodic CNN", "Enc.-Dec. CNN", "FNO", "DeepONet", "FDM", "FEM", "Spectral", "ADANNs"]
space_grid = x_values(nr_spacediscr, space_size)

evaluate_and_plot(methods, 
                  methods_data, 
                  method_categories, 
                  test_initial_values_rough, 
                  test_ref_sol_rough, 
                  space_grid, 
                  space_size, 
                  output_folder_dir, 
                  pde_name, 
                  dim=dim, 
                  nr_of_eval_runs=nr_of_eval_runs, 
                  plot_histogram=plot_histogram,
                  legend_loc=None,
                  nr_of_plots=1 if test_run else 3
                  )

#Save all the data in an Excel sheet
local_vars = locals()
params_dict = {k: [v] for k, v in local_vars.items() if isinstance(v, (int, str, float)) and k[0] != '_'}
save_excel_sheet(methods_data, params_dict, output_folder_dir + f"Results_{pde_name}.xlsx")

In [ ]:
create_error_vs_comptime_plot(method_categories, output_folder_dir, pde_name)